# Children Act disposals csv

### This code summarises the information produced from the children act disposals code into tidy data sets, and combines them all at the end.
1. Child counts
2. Order counts
3. Case counts
4. Combined orders (SGO and SO)
5. Combine all datasets

In [ ]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

### CSV creation stages

#### Child counts

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) AS Quarter,
  'Disposal' AS Type,
  'Children' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
   Disp_type AS Disposal_type,
   order_desc AS Order_type,
   order_code AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_disps_all_children
Where
  Year>=2011
Group BY
  Year,
  Quarter,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End,
  Disp_type,
  order_desc,
  order_code
  
""",

"child_counts")

### Order counts

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) AS Quarter,
  'Disposal' AS Type,
  'Order type' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
   Disp_type AS Disposal_type,
   order_desc AS Order_type,
   order_code AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_all_disposals
Where
  Year>=2011
Group BY
  Year,
  Quarter,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End,
  Disp_type,
  order_desc,
  order_code
  
""",

"order_type_counts")

### Case counts

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) AS Quarter,
  'Cases' AS Type,
  'Cases closed' AS Count_type,
  CASE WHEN case_type = 'C' THEN 'Public law'
       WHEN case_type = 'P' THEN 'Private law'
    End AS Public_private,
   'N/A' AS Disposal_type,
   'All' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_closed_cases
Where
  Year>=2011
Group BY
  Year,
  Quarter,
  CASE WHEN case_type = 'C' THEN 'Public law'
       WHEN case_type = 'P' THEN 'Private law'
    End
  
""",

"disp_case_counts")

### Combined Special Guardianship and Supervision orders (this code will need revisiting once duplicate issues have been resolved)

In [ ]:
#Supervision orders
pydb.create_temp_table( 
f"""
SELECT * FROM __temp__.ca_disps_all_children WHERE order_code = 4   
""",
"so")

In [ ]:
#SGO orders
pydb.create_temp_table( 
f"""
SELECT * FROM __temp__.ca_disps_all_children WHERE order_code = 27   
""",
"sgo")

In [ ]:
#Join SO and SGO info together to see if both orders were given to the same child on the same day
pydb.create_temp_table( 
f"""
SELECT
  so.*,
  sgo.order_desc AS sgo_order
FROM
  __temp__.so so 
  LEFT JOIN __temp__.sgo sgo 
    ON so.receipt_date = sgo.receipt_date
    AND so.child_role_id = sgo.child_role_id
WHERE
  sgo.order_desc IS NOT NULL
""",

"so_sgo_children")

In [ ]:
# now get rid of the children and count only orders
pydb.create_temp_table( 
f"""
SELECT
  DISTINCT year,
           quarter,
           order_case_type,
           disp_type_code,
           event
FROM
  __temp__.so_sgo_children

""",

"so_sgo_orders")

In [ ]:
# csv data for combined orders - children
pydb.create_temp_table( 
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) AS Quarter,
  'Disposal' AS Type,
  'Combined orders - children' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
   'Combined orders' AS Disposal_type,
   'Supervision and SGO' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.so_sgo_children
Where
  Year>=2011
  AND disp_type_code IN (1,2) 
Group BY
  Year,
  Quarter,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End
  
""",

"combined_orders_CH")

In [ ]:
#csv data for combined orders - orders made only
pydb.create_temp_table( 
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) AS Quarter,
  'Disposal' AS Type,
  'Combined orders - orders' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
   'Combined orders' AS Disposal_type,
   'Supervision and SGO' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.so_sgo_orders
Where
  Year>=2011
  AND disp_type_code IN (1,2) 
Group BY
  Year,
  Quarter,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End
  
""",

"combined_orders_ORD")

### Combine all disposal datasets

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  *
FROM
  __temp__.child_counts
UNION ALL
SELECT
  *
FROM
  __temp__.order_type_counts
UNION ALL
SELECT
  *
FROM
  __temp__.disp_case_counts
UNION ALL 
SELECT
  *
FROM
  __temp__.combined_orders_CH
UNION ALL 
SELECT
  *
FROM
  __temp__.combined_orders_ORD 
  
""",

"ca_disp_csv")